In [13]:
import os
from glob import glob
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from pprint import pprint
import shutil

In [14]:
chartqa = {
    'notextBar': {
        'answer': '13942',
        'image_path': 'two_col_40186.png',
        'options': ['3194', '13942', '14495', '28466'],
        'question': "How many deaths resulted from cocaine use?"
    },
    'notextLine': {
        'answer': '11.8%',
        'image_path': 'two_col_4524.png',
        'options': ['11.8%', '12.4%', '14.3%', '17%'],
        'question': 'What is the poverty rate in California in the year 2019?'
    },
    'notextPie': {
        'answer': '12%',
        'image_path': 'two_col_61107.png',
        'options': ['9%', '12%', '29%', '50%'],
        'question': 'What is the percentage of "Very important"?'
    },
    'notextStackedBar': {
        'answer': 'False',
        'image_path': 'multi_col_60949.png',
        'options': ['True', 'False'],
        'question': 'The Y axis represents the date'
    },
    'textBar': {
        'answer': 'Ireland',
        'image_path': '05411753006467.png',
        'options': ['Dominican Republic', 'Ireland', 'Guadeloupe'],
        'question': 'Which country had the second largest value?'
    },
    'textLine': {
        'answer': '1974',
        'image_path': '19371621021871.png',
        'options': ['1964', '1970', '1974', '1980'],
        'question': 'In which year did public trust in government first drop below 50%?'
    },
    'textPie': {
        'answer': 'True',
        'image_path': '43.png',
        'options': ['True', "False"],
        'question': "Percentage of people who 'Do not use Latinx' is 20%"
    },
    'textStackedBar': {
        'answer': '',
        'image_path': '9280.png',
        'options': ['17', '24', '40', '57'],
        'question': "What's the percentage of people choosing 'Very' in Feb 2016?"
    }
}

In [15]:
vlat_data = {
    "TreeMap": {
        "question": "The number of unique visitors for Amazon was more than that of Yahoo in 2010.",
        "answer": "False",
        "image_path": "TreeMap.png",
        "options": ["True", "False"]
    },
    "Stacked100": {
        "question": "The percentage of Gold medals of Australia is lower than Japan",
        "answer": "True",
        "image_path": "Stacked100.png",
        "options": ["True", "False"]
    },
    "Histogram": {
        "question": "More people traveled between 10 and 20km than between 60 and 70km.",
        "answer": "True",
        "image_path": "Histogram.png",
        "options": ["True", "False"]
    },
    "Choropleth": {
        "question": "In which state has the highest value?",
        "answer": "Nevada (NV)",
        "image_path": "Choropleth.png",
        "options": ["Michigan (MI)", "Illinois (IL)", "Nevada (NV)", "New York (NY)"]
    },
    "PieChart": {
        "question": "In which company is the global smartphone market share the smallest?",
        "answer": "Vivo",
        "image_path": "PieChart.png",
        "options": ["Samsung", "Apple", "Vivo", "Xiaomi"]
    },
    "BubbleChart": {
        "question": "The length of metro systems in this chart ranges from ______",
        "answer": "160 - 460 km",
        "image_path": "BubbleChart.png",
        "options": ["160 - 460 km", "240 - 380 km", "240 - 560 km", "180 - 560 km"]
    },
    "StackedBar": {
        "question": "The cost of vodka in Toronto is higher than that of Zurich.",
        "answer": "True",
        "image_path": "StackedBar.png",
        "options": ["True", "False"]
    },
    "LineChart": {
        "question": "Over the first three months in 2020, the price of a barrel of oil was ____________.",
        "answer": "falling",
        "image_path": "LineChart.png",
        "options": ["rising", "falling", "staying the same"]
    },
    "BarChart": {
        "question": "How many countries have a faster average internet speed than Taiwan?",
        "answer": "4",
        "image_path": "BarChart.png",
        "options": ["3", "4", "5", "6"]
    },
    "AreaChart": {
        "question": "Between July 2018 and July 2019, the average price of a pound of coffee beans ranged from _______ to ________",
        "answer": "$0.71 - $0.87",
        "image_path": "AreaChart.png",
        "options": ["$0.61 - $0.75", "$0.75 - $0.85", "$0.71 - $0.87", "$0.76 - $0.85"]
    },
    "StackedArea": {
        "question": "In the UK, the number of girls named \"Amelia\" was higher in 2014 than in 2013",
        "answer": "False",
        "image_path": "StackedArea.png",
        "options": ["True", "False"]
    },
    "Scatterplot": {
        "question": "What is the weight for the person who is 161cm tall?",
        "answer": "44 kg",
        "image_path": "Scatterplot.png",
        "options": ["44 kg", "52 kg", "61 kg", "67 kg"]
    }
}

additional_data = {
   "TreeMap": {
        "question": "Samsung is nested in the Financial category.",
        "answer": "False",
        "image_path": "VLAT-TreeMap.png",
        "options": ["True", "False"]
    },
    "Stacked100": {
        "question": "What is the education level of people in which the Democrats have the lowest approval rating?",
        "answer": "College Graduate",
        "image_path": "VLAT-Stacked100.png",
        "options": ["High School Graduate or Less", "Some College Degree", "College Graduate", "Postgraduate study"]
    },
    "Histogram": {
        "question": "Most people rated the taxi service between ______ and ______ stars",
        "answer": "4.4 - 4.6",
        "image_path": "VLAT-Histogram.png",
        "options": ["4.2 - 4.4", "4.4 - 4.6", "4.6 - 4.8", "4.8 - 5.0"]
    },
    "Choropleth": {
        "question": "In 2015, the unemployment rate for Wisconsin (WI) was higher than that of Washington (WA).",
        "answer": "False",
        "image_path": "VLAT-Choropleth.png",
        "options": ["true", "False"]
    },
    "PieChart": {
        "question": "The global smartphone market share of Apple is larger than that of Huawei.",
        "answer": "True",
        "image_path": "VLAT-PieChart.png",
        "options": ["True", "False"]
    },
    "BubbleChart": {
        "question": "Which city's metro system has the largest number of stations?",
        "answer": "New York City",
        "image_path": "VLAT-BubbleChart.png",
        "options": ["Seoul", "Beijing", "New York City", "Shanghai"]
    },
    "StackedBar": {
        "question": "What is the cost of peanuts in Las Vegas?",
        "answer": "$12",
        "image_path": "VLAT-StackedBar.png",
        "options": ["$12", "$16.7", "$23.4", "$35.4"]
    },
    "LineChart": {
        "question": "What was the price of a barrel of oil in February 2015?",
        "answer": "$50.24",
        "image_path": "VLAT-LineChart.png",
        "options": ["$57.36", "$47.82", "$50.24", "$39.72"]
    },
    "BarChart": {
        "question": "What is the average internet speed in Japan?",
        "answer": "15 Mbps",
        "image_path": "VLAT-BarChart.png",
        "options": ["10 Mbps", "14 Mbps", "15 Mbps", "16 Mbps"]
    },
    "AreaChart": {
        "question": "What was the average price of a pound of coffee beans in September 2013?",
        "answer": "$5.1",
        "image_path": "VLAT-AreaChart.png",
        "options": ["$4.9", "$5.0", "$5.1", "$5.2"]
    },
    "StackedArea": {
        "question": "Approximately, what is the ratio of the number of girls named 'Olivia' to those named 'Isla' in 2014 in the UK?",
        "answer": "1 to 1",
        "image_path": "VLAT-StackedArea.png",
        "options": ["1 to 1", "1 to 2", "1 to 3", "1 to 4"]
    },
    "Scatterplot": {
        "question": "There is a positive linear relationship between the height and the weight of the 85 males.",
        "answer": "True",
        "image_path": "VLAT-Scatterplot.png",
        "options": ["True", "False"]
    }
}

In [17]:
def static_grids(img, grid_count=8):
    image_height, image_width = img.shape[:2]
    interval_y = int(np.ceil(image_height / (grid_count+1)))
    interval_x = int(np.ceil(image_width / (grid_count+1)))
    x_grids = []
    y_grids = []
    
    for i in range(interval_x, image_width, interval_x):
        x_grids.append(i)
    for i in range(interval_y, image_height, interval_y):
        y_grids.append(i)
    
    return x_grids, y_grids

In [18]:

def create_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "literacy/assets/bubble.jsx",
            "instructionLocation": "sidebar",
            "nextButtonLocation": "sidebar",
            "nextButtonDisableTime": 60000,
            "nextButtonEnableTime": 5000
        },
    }

In [24]:

def create_default_components():
    return {
        "instructions": {
            "type": "markdown",
            "path": "literacy/assets/instructions.md",
            "response": []
        },  
        "consent": {
            "type": "markdown",
            "path": "shared/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "vlat_intro": {
            "type": "markdown",
            "path": "literacy/assets/vlat_intro.md",
            "response": []
        },
        "demographics": {
            "type": "markdown",
            "path": "shared/blank.md",
            "response": [
                {
                    "id": "gender",
                    "prompt": "What is your **gender**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Woman",
                        "Man",
                        "Prefer not to say"
                    ],
                    "withDivider": True

                },
                {
                    "id": "age",
                    "prompt": "What is your **age**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "Under 18 years",
                        "18-24 years",
                        "25-34 years",
                        "35-44 years",
                        "45-54 years",
                        "55-64 years",
                        "65 years or older",
                        "Prefer not to say"
                    ],
                    "withDivider": True
                },
                {
                    "id": "education",
                    "prompt": "What is the **highest degree or level of education** you have completed?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Less than high school",
                        "High school diploma or equivalent",
                        "Bachelor's degree or equivalent",
                        "Master's degree or equivalent",
                        "Doctoral degree or equivalent"
                    ],
                    "withDivider": True
                }
            ]
        },
    }


def create_bubble_components(data, tag='basic'):
    tasks = dict()
    for chart_type, value in data.items():
        question = value["question"]
        image_path = value["image_path"]
        options = value["options"]
        answer = value["answer"]

        tasks[f"{chart_type}-{tag}"] = {
            "baseComponent": "bubble",
            "instruction": f" **Q: {question}**",
            "parameters": {
                "image": f"../literacy/assets/images/{image_path}",
                "question": question,
                "radius": 32,
            },
            "response": [
                {
                    "id": f"{chart_type}-{tag}",
                    "prompt": "Your answer",
                    "required": True,
                    "location": "sidebar",
                    "type": "radio",
                    "options": options + ["Skip"]
                },
            ],
            "correctAnswer": [
                {
                "id": f"{chart_type}-{tag}",
                "answer": answer
                }
            ]
        }

    return tasks

In [43]:
batch1 = ["notextBar-chartqa", "notextLine-chartqa", "notextPie-chartqa", "notextStackedBar-chartqa", 
          "TreeMap-same_inst", "Stacked100-same_inst", "Histogram-same_inst", "Choropleth-same_inst", "PieChart-same_inst", "BubbleChart-same_inst", 
          "TreeMap-diff_inst", "Stacked100-diff_inst", "Histogram-diff_inst", "Choropleth-diff_inst", "PieChart-diff_inst", "BubbleChart-diff_inst"]
batch2 = ["textBar-chartqa", "textLine-chartqa", "textPie-chartqa", "textStackedBar-chartqa", 
          "StackedBar-same_inst", "LineChart-same_inst", "BarChart-same_inst", "AreaChart-same_inst", "StackedArea-same_inst", "Scatterplot-same_inst",
          "StackedBar-diff_inst", "LineChart-diff_inst", "BarChart-diff_inst", "AreaChart-diff_inst", "StackedArea-diff_inst", "Scatterplot-diff_inst"]

In [44]:
def sequence_generator():
    sequence = {
        "order": "fixed",
        "components": [
            "consent",
            "vlat_intro",
            "$mini-vlat.se.full",
            "instructions",
            {
                "id": "batch",
                "order": "latinSquare",
                "numSamples": 1,
                "components": [
                    {
                        "id": "batch1",
                        "order": "latinSquare",
                        "components": batch1
                    },
                    {
                        "id": "batch2",
                        "order": "latinSquare",
                        "components": batch2
                    }
                ]
            },
            "demographics"
        ]
    }
    return sequence

In [45]:
prolificRedirection = "https://app.prolific.com/submissions/complete?cc=C1DEBJ8K"

In [46]:
same_inst_components = create_bubble_components(vlat_data, 'same_inst')
diff_inst_components = create_bubble_components(additional_data, 'diff_inst')
chartqa_components = create_bubble_components(chartqa, 'chartqa')

In [47]:
default_components = create_default_components()
baseComponents = create_base_components()
components = default_components | same_inst_components | diff_inst_components | chartqa_components
sequence = sequence_generator()
print(f"Total number of components: {len(components)}")

Total number of components: 36


In [48]:
with open("config.json", "r") as f:
    config = json.load(f)
config['uiConfig']['studyEndMsg'] = f"**Thank you for completing the study. You may click this link and return to Prolific**: [{prolificRedirection}]({prolificRedirection})"
config['components'] = components
config['sequence'] = sequence
config['baseComponents'] = baseComponents
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)